<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Primarly_VitalDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install pyvital vitaldb vital
!pip install neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.7/824.7 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 109.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 6

In [33]:

import vitaldb
import neurokit2 as nk
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [34]:

Hemodynamic_Parameters = [
'Solar8000/ST_II',
'Solar8000/ST_VI',
'Solar8000/ART_DBP',
'Solar8000/ART_SBP'
]

In [35]:
SRATE = 100
vals = vitaldb.load_case(1, Hemodynamic_Parameters, 1 / SRATE)

In [44]:
type(str(vals[0][0]))

str

In [37]:
MAP = []
for i in tqdm(range(vals.shape[0])):
  MAP.append([(vals[i][3] + 2 * vals[i][2])/3])

new_vals = np.append(vals, MAP ,axis = 1)

  0%|          | 0/1103281 [00:00<?, ?it/s]

In [45]:
for i in range(new_vals.shape[0]):
  if str(new_vals[i][0]) != 'nan':
    print(new_vals[i])

Streaming output truncated to the last 5000 lines.
[-0.60000002         nan 44.         97.         61.66666667]
[ -0.5                 nan  44.         100.          62.66666667]
[-0.5  nan 44.  98.  62. ]
[-0.5  nan 44.  98.  62. ]
[-0.5                nan 43.         98.         61.33333333]
[-0.5                nan 44.         97.         61.66666667]
[-0.5  nan 45.  99.  63. ]
[-0.5                nan 44.         97.         61.66666667]
[-0.5                nan 45.         97.         62.33333333]
[-0.5  nan 44.  98.  62. ]
[-0.5                nan 44.         97.         61.66666667]
[-0.5  nan 44.  98.  62. ]
[ -0.5   nan  37.  100.   58. ]
[-0.5                nan 40.         98.         59.33333333]
[-0.5                nan 42.         98.         60.66666667]
[-0.40000001         nan 43.         96.         60.66666667]
[ -0.40000001          nan  38.         105.          60.33333333]
[-0.40000001         nan 43.         96.         60.66666667]
[-0.40000001         nan 45.

In [ ]:
for i in range(2000):
  print(i,vals[i])

In [ ]:
import pandas as pd

df = pd.DataFrame(vals)
df.columns = Hemodynamic_Parameters
df["MAP"] = np.nan
df["Index"] = np.nan

In [ ]:
df.shape

(1103281, 6)

In [ ]:
count=1

for i in tqdm(range(df.shape[0])):
  df["MAP"][i] = (df["Solar8000/ART_SBP"][i] + 2 * df["Solar8000/ART_DBP"][i])/3
  df["Index"] = count
  count+=1

  0%|          | 0/1103281 [00:00<?, ?it/s]

<ipython-input-34-4346cd95e4cc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MAP"][i] = (df["Solar8000/ART_SBP"][i] + 2 * df["Solar8000/ART_DBP"][i])/3


In [ ]:
df.head()

numpy.float64

In [ ]:

df = df.query("MAP.notna()")

,Solar8000/ST_II,Solar8000/ST_VI,Solar8000/ART_DBP,Solar8000/ART_SBP,MAP
181,-0.2,NaN,-8.0,-8.0,-8.000000
381,-0.2,NaN,-8.0,-8.0,-8.000000
581,-0.2,NaN,-8.0,-8.0,-8.000000
781,-0.4,NaN,-9.0,-9.0,-9.000000
981,-0.4,NaN,-9.0,-9.0,-9.000000
...,...,...,...,...,...
1090280,-1.3,NaN,107.0,199.0,137.666667
1090480,-1.3,NaN,106.0,196.0,136.000000
1090680,-1.3,NaN,101.0,198.0,133.333333
1090880,-1.3,NaN,105.0,151.0,120.333333


In [ ]:
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # read track list
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # read case information


In [ ]:
list(df_trks['tname'])